In [1]:
import fairing
from fairing import TrainJob
from fairing.backends import KubeflowGKEBackend
from fairing.kubernetes.utils import get_resource_mutator

## Executing a python file

In [9]:
%%writefile train.py
print("hello world!")

Overwriting train.py


In [10]:
job = TrainJob("train.py", backend=KubeflowGKEBackend())
job.submit()

Using default docker registry: gcr.io/caip-dexter-bugbash/fairing-job
Using default base docker image: registry.hub.docker.com/library/python:3.7.2
Using builder: <class 'fairing.builders.append.append.AppendBuilder'>
Building the docker image.
Building image using Append builder...
Creating docker context: /tmp/fairing_context_5xhzxv7o
Loading Docker credentials for repository 'registry.hub.docker.com/library/python:3.7.2'
Image successfully built in 3.302749749999748s.
Pushing image gcr.io/caip-dexter-bugbash/fairing-job/fairing-job:AF73294F...
Loading Docker credentials for repository 'gcr.io/caip-dexter-bugbash/fairing-job/fairing-job:AF73294F'
Invoking 'docker-credential-gcloud' to obtain Docker credentials.
Successfully obtained Docker credentials.
Uploading gcr.io/caip-dexter-bugbash/fairing-job/fairing-job:AF73294F
Layer sha256:7d3cdae560215b5913fcb23622a9f98b73109c422da51fff1d9eab3c53f0756b exists, skipping
Layer sha256:d54db43011fd116b8cb6d9e49e268cee1fa6212f152b30cbfa7f3c4c6

hello world!


'fairing-job-pjmq5'

## Executing a python function

In [12]:
def train():
    print("simple train job!")

In [21]:
train()

CPU count: 12
Memory: {} 32.0


In [22]:
job = TrainJob(train, backend=KubeflowGKEBackend())
job.submit()

Using default docker registry: gcr.io/caip-dexter-bugbash/fairing-job
Using default base docker image: registry.hub.docker.com/library/python:3.7.2
Using builder: <class 'fairing.builders.append.append.AppendBuilder'>
Building the docker image.
Building image using Append builder...
Creating docker context: /tmp/fairing_context_5fuymnhx
/Users/cartick/Documents/workspace/fairing/venv/lib/python3.7/site-packages/fairing/__init__.py already exists in Fairing context, skipping...
Loading Docker credentials for repository 'registry.hub.docker.com/library/python:3.7.2'
Image successfully built in 1.7307556389996535s.
Pushing image gcr.io/caip-dexter-bugbash/fairing-job/fairing-job:C6D1D8A1...
Loading Docker credentials for repository 'gcr.io/caip-dexter-bugbash/fairing-job/fairing-job:C6D1D8A1'
Invoking 'docker-credential-gcloud' to obtain Docker credentials.
Successfully obtained Docker credentials.
Uploading gcr.io/caip-dexter-bugbash/fairing-job/fairing-job:C6D1D8A1
Layer sha256:7d3cdae5

CPU count: 8
Memory: {} 29.450111389160156


'fairing-job-pq8jh'

## Executing a complete notebook

In [14]:
%%writefile requirements.txt
papermill
jupyter

Overwriting requirements.txt


In [15]:
job = TrainJob("train.ipynb", backend=KubeflowGKEBackend(), input_files=["requirements.txt"])
job.submit()

Using default docker registry: gcr.io/caip-dexter-bugbash/fairing-job
Using default base docker image: registry.hub.docker.com/library/python:3.7.2
Using builder: <class 'fairing.builders.docker.docker.DockerBuilder'>
Building the docker image.
Building image using docker
Docker command: ['papermill', 'train.ipynb', 'fairing_output_notebook.ipynb', '--log-output']
Creating docker context: /tmp/fairing_context_3mycavj9
Building docker image gcr.io/caip-dexter-bugbash/fairing-job/fairing-job:28265AFC...
Build output: Step 1/7 : FROM registry.hub.docker.com/library/python:3.7.2
Build output: 
Build output: ---> 2053ca75899e
Build output: Step 2/7 : WORKDIR /app/
Build output: 
Build output: ---> Using cache
Build output: ---> 2c7f8a216fa8
Build output: Step 3/7 : ENV FAIRING_RUNTIME 1
Build output: 
Build output: ---> Using cache
Build output: ---> 13640ae5453f
Build output: Step 4/7 : COPY /app//requirements.txt /app/
Build output: 
Build output: ---> Using cache
Build output: ---> 3161c

Input Notebook:  train.ipynb
Output Notebook: fairing_output_notebook.ipynb
Executing notebook with kernel: python3
Executing Cell 1---------------------------------------
training in notebook!

Ending Cell 1------------------------------------------


Cleaning up job fairing-job-2qj98...


'fairing-job-2qj98'

## Executing it with large #CPUs and huge Memory
You Kubernetes cluster should have a node pool that can satisfy these resource requests. For example, to schedule a job with 90 cpus and 600GB memory you need a nodepool created using n1-hihmem-624 in GCP.

In [16]:
import multiprocessing
import os
def train():
    print("CPU count: {}".format(multiprocessing.cpu_count()))
    print("Memory: {}", os.sysconf('SC_PAGE_SIZE') * os.sysconf('SC_PHYS_PAGES')/(1024.**3))
train()

CPU count: 12
Memory: {} 32.0


In [17]:
job = TrainJob(train, backend=KubeflowGKEBackend(), 
              pod_spec_mutators=[get_resource_mutator(cpu=90, memory=600)])
job.submit()

Using default docker registry: gcr.io/caip-dexter-bugbash/fairing-job
Using default base docker image: registry.hub.docker.com/library/python:3.7.2
Using builder: <class 'fairing.builders.append.append.AppendBuilder'>
Building the docker image.
Building image using Append builder...
Creating docker context: /tmp/fairing_context_desyj_od
/Users/cartick/Documents/workspace/fairing/venv/lib/python3.7/site-packages/fairing/__init__.py already exists in Fairing context, skipping...
Loading Docker credentials for repository 'registry.hub.docker.com/library/python:3.7.2'
Image successfully built in 1.739382934000787s.
Pushing image gcr.io/caip-dexter-bugbash/fairing-job/fairing-job:4C228EA7...
Loading Docker credentials for repository 'gcr.io/caip-dexter-bugbash/fairing-job/fairing-job:4C228EA7'
Invoking 'docker-credential-gcloud' to obtain Docker credentials.
Successfully obtained Docker credentials.
Uploading gcr.io/caip-dexter-bugbash/fairing-job/fairing-job:4C228EA7
Layer sha256:7d3cdae56

CPU count: 96
Memory: {} 614.1527938842773


'fairing-job-k2d5p'

## Executing with a GPU 

In [18]:
from kubernetes.client.models.v1_resource_requirements import V1ResourceRequirements
def add_gpu(kube_manager, pod_spec, namespace):
    pod_spec.containers[0].resources = V1ResourceRequirements(limits={"nvidia.com/gpu":"1"})

In [19]:
%%writefile gpu_test.py
import subprocess
def train():
    print(subprocess.check_output(["nvidia-smi"]))
train()

Overwriting gpu_test.py


In [20]:
job = TrainJob("gpu_test.py",
               base_docker_image="tensorflow/tensorflow:2.0.0b0-gpu",
               backend=KubeflowGKEBackend(),
               pod_spec_mutators=[add_gpu])
job.submit()

Using default docker registry: gcr.io/caip-dexter-bugbash/fairing-job
Using builder: <class 'fairing.builders.append.append.AppendBuilder'>
Building the docker image.
Building image using Append builder...
Creating docker context: /tmp/fairing_context_k2b58s0h
Loading Docker credentials for repository 'tensorflow/tensorflow:2.0.0b0-gpu'
Image successfully built in 2.1183637859994633s.
Pushing image gcr.io/caip-dexter-bugbash/fairing-job/fairing-job:BA655542...
Loading Docker credentials for repository 'gcr.io/caip-dexter-bugbash/fairing-job/fairing-job:BA655542'
Invoking 'docker-credential-gcloud' to obtain Docker credentials.
Successfully obtained Docker credentials.
Uploading gcr.io/caip-dexter-bugbash/fairing-job/fairing-job:BA655542
Layer sha256:5788fbc934ee33c1d7f95fb77587eabf716f2e5884b2c02c8a9c329b3629dc39 exists, skipping
Layer sha256:5138351ecc1c8cb2633c072222751a972edc349ff9e025ba217a64ff0f41dd6e exists, skipping
Layer sha256:e059dd98ac7cff88cacd4e01f2f1d56af872618aac98b0aff8

Fri Jun 14 02:16:44 2019
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.79       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 300W |      0MiB / 16130MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+

+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU       PID   Type   Process name                             Usage      |
|=============

'fairing-job-tjq9k'

## MNIST on GPU

In [5]:
%%writefile mnist.py
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow_datasets as tfds
import tensorflow as tf

BUFFER_SIZE = 10000
BATCH_SIZE = 64

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

datasets, info = tfds.load(name='mnist',
                           with_info=True,
                           as_supervised=True)
train_datasets_unbatched = datasets['train'].map(scale).shuffle(BUFFER_SIZE)
train_datasets = train_datasets_unbatched.batch(BATCH_SIZE)

def build_and_compile_cnn_model():
    model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
      metrics=['accuracy'])
    return model

single_worker_model = build_and_compile_cnn_model()
single_worker_model.fit(x=train_datasets, epochs=3)

Overwriting mnist.py


In [6]:
%%writefile requirements.txt
tensorflow_datasets

Overwriting requirements.txt


In [8]:
job = TrainJob("mnist.py",
               base_docker_image="tensorflow/tensorflow:2.0.0b0-gpu",
               backend=KubeflowGKEBackend(),
               pod_spec_mutators=[add_gpu],
               input_files=["requirements.txt"])
job.submit()

Using default docker registry: gcr.io/caip-dexter-bugbash/fairing-job
Using builder: <class 'fairing.builders.docker.docker.DockerBuilder'>
Building the docker image.
Building image using docker
Docker command: ['python', '/app/mnist.py']
Creating docker context: /tmp/fairing_context_i7oq20xv
Building docker image gcr.io/caip-dexter-bugbash/fairing-job/fairing-job:CEDDE73E...
Build output: Step 1/7 : FROM tensorflow/tensorflow:2.0.0b0-gpu
Build output: 
Push output: Pulling from tensorflow/tensorflow None
Push output: Pulling fs layer None
Push output: Pulling fs layer None
Push output: Pulling fs layer None
Push output: Pulling fs layer None
Push output: Pulling fs layer None
Push output: Pulling fs layer None
Push output: Pulling fs layer None
Push output: Pulling fs layer None
Push output: Pulling fs layer None
Push output: Pulling fs layer None
Push output: Pulling fs layer None
Push output: Pulling fs layer None
Push output: Pulling fs layer None
Push output: Pulling fs layer None

Push output: Downloading [==>                                                ]  7.503MB/138.6MB
Push output: Extracting [============================================>      ]  25.66MB/28.86MB
Push output: Downloading [========>                                          ]  11.24MB/64.04MB
Push output: Downloading [=>                                                 ]  26.27MB/665.9MB
Push output: Extracting [===============================================>   ]  27.43MB/28.86MB
Push output: Downloading [===>                                               ]  8.584MB/138.6MB
Push output: Downloading [==========>                                        ]  12.84MB/64.04MB
Push output: Downloading [==>                                                ]  27.88MB/665.9MB
Push output: Extracting [=================================================> ]  28.61MB/28.86MB
Push output: Downloading [===>                                               ]  9.653MB/138.6MB
Push output: Downloading [==========>      

Push output: Downloading [=====>                                             ]  70.27MB/665.9MB
Push output: Downloading [====================>                              ]  26.23MB/64.04MB
Push output: Downloading [===========>                                       ]  31.12MB/138.6MB
Push output: Pull complete None
Push output: Downloading [=====>                                             ]  72.96MB/665.9MB
Push output: Downloading [===========>                                       ]  31.64MB/138.6MB
Push output: Downloading [=====>                                             ]  76.16MB/665.9MB
Push output: Downloading [====================>                              ]  26.76MB/64.04MB
Push output: Downloading [=====>                                             ]  78.85MB/665.9MB
Push output: Downloading [===========>                                       ]  32.18MB/138.6MB
Push output: Downloading [======>                                            ]  82.05MB/665.9MB
Push out

Push output: Downloading [================>                                  ]  223.2MB/665.9MB
Push output: Downloading [=========================>                         ]  32.67MB/64.04MB
Push output: Downloading [================>                                  ]  225.4MB/665.9MB
Push output: Downloading [===============>                                   ]  41.87MB/138.6MB
Push output: Downloading [=================>                                 ]  228.6MB/665.9MB
Push output: Downloading [=========================>                         ]  33.21MB/64.04MB
Push output: Downloading [=================>                                 ]  231.8MB/665.9MB
Push output: Downloading [=================>                                 ]  234.5MB/665.9MB
Push output: Downloading [=================>                                 ]  237.7MB/665.9MB
Push output: Downloading [===============>                                   ]  42.41MB/138.6MB
Push output: Downloading [==============

Push output: Downloading [==============================>                    ]  39.61MB/64.04MB
Push output: Downloading [==============================>                    ]  410.5MB/665.9MB
Push output: Downloading [=================>                                 ]  48.35MB/138.6MB
Push output: Downloading [===============================>                   ]  413.7MB/665.9MB
Push output: Downloading [===============================>                   ]  416.9MB/665.9MB
Push output: Downloading [===============================>                   ]  420.1MB/665.9MB
Push output: Downloading [===============================>                   ]  423.3MB/665.9MB
Push output: Downloading [===============================>                   ]    426MB/665.9MB
Push output: Downloading [===============================>                   ]  40.14MB/64.04MB
Push output: Downloading [=================>                                 ]  48.89MB/138.6MB
Push output: Downloading [==============

...

Reading...: 0 examples [00:00, ? examples/s]
                                            
Writing...:   0%|          | 0/6000 [00:00<?, ? examples/s]
                                                           
Reading...: 0 examples [00:00, ? examples/s]
                                            
Shuffling...:  60%|██████    | 6/10 [00:00<00:00, 14.50 shard/s]
Reading...: 0 examples [00:00, ? examples/s]
                                            
Writing...:   0%|          | 0/6000 [00:00<?, ? examples/s]
                                                           
Reading...: 0 examples [00:00, ? examples/s]
                                            
Shuffling...:  80%|████████  | 8/10 [00:00<00:00, 12.90 shard/s]
Reading...: 0 examples [00:00, ? examples/s]
                                            
Writing...:   0%|          | 0/6000 [00:00<?, ? examples/s]
                                                           
Reading...: 0 examples [00:00, ? examples/s]
               

    792/Unknown - 20s 26ms/step - loss: 2.1883 - accuracy: 0.2990
    800/Unknown - 20s 25ms/step - loss: 2.1867 - accuracy: 0.3011
    808/Unknown - 20s 25ms/step - loss: 2.1850 - accuracy
    816/Unknown - 20s 25ms/step -
    824/U
    831/Unknown - 20s 25ms/step - loss: 2.1801 - accuracy: 0.3093
    839/Unknown - 20s 24ms/step - loss: 2.1783 - accuracy: 0.3115
    847/Unknown - 21s 24ms/step - loss: 2.1765 - accuracy: 0.3136
    855/Unknown - 21s 24ms/step - loss: 2.17
    863/Unknown - 21
    870/Unknown - 21s 24ms/step - loss: 2.1714 - accuracy: 0.3193
    878/Unknown - 21s 24ms/step - loss: 2.1695 - accuracy: 0.3214
    886/Unknown - 21s 23ms/step - loss: 2.1674 - accuracy: 0.3235
    894/Unknown - 21s 23ms/step - loss: 2.1655 - accu

Epoch 2/3
938/938 [==============================] - 18s 19ms/step - loss: 1.4367 - accuracy: 0.7020
Epoch 3/3
938/938 [==============================] - 17s 18ms/step - loss: 0.7376 - accuracy: 0.8302


Cleaning up job fairing-job-d9rt9...


'fairing-job-d9rt9'